In [1]:
import pandas as pd

df = pd.read_csv('data/Nifty50Scrips.csv')

stock_ids = df['Symbol']


In [2]:
stock_ids

0      POWERGRID
1       JSWSTEEL
2      COALINDIA
3           NTPC
4     ADANIPORTS
5      BRITANNIA
6       DIVISLAB
7      TATASTEEL
8     BHARTIARTL
9     TATAMOTORS
10     SUNPHARMA
11      HINDALCO
12     EICHERMOT
13    BAJAJFINSV
14            LT
15          SBIN
16         CIPLA
17           UPL
18           IOC
19      RELIANCE
20    TATACONSUM
21     NESTLEIND
22         TECHM
23     KOTAKBANK
24      HDFCLIFE
25    BAJFINANCE
26          BPCL
27       HCLTECH
28       SBILIFE
29       DRREDDY
30           TCS
31           ITC
32    ULTRACEMCO
33    HINDUNILVR
34          ONGC
35      SHREECEM
36          HDFC
37     ICICIBANK
38      HDFCBANK
39    BAJAJ-AUTO
40    HEROMOTOCO
41        GRASIM
42         WIPRO
43           M&M
44        MARUTI
45         TITAN
46      AXISBANK
47          INFY
48    INDUSINDBK
49    ASIANPAINT
Name: Symbol, dtype: object

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if 'Dec 2021' in test_list:
        quarter = 'Dec 2021'
    else:
        quarter = 'Sep 2021'

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-5)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, data_source='yahoo', start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

POWERGRID
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11

[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
BHARTIARTL
162
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*****

[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
CIPLA
162
2019-02-15 00:00:00
[**********

[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
KOTAKBANK
188
2019-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[******

[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TCS.NS: Data doesn't exist for startDate = 1644431400, endDate = 1644863400
ITC
162
2019-02-15 00:00:00
[***

[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
HDFCBANK
164
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*******

[*********************100%***********************]  1 of 1 completed
2019-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2019-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[********************

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
6,POWERGRID.NS,2020-08-15,131.55,2.94,9457.0,15.21,38020.0,8.65
7,POWERGRID.NS,2020-11-15,141.00,4.44,9530.0,15.96,38499.0,8.83
8,POWERGRID.NS,2021-02-15,159.49,4.83,10142.0,16.96,39277.0,9.40
9,POWERGRID.NS,2021-05-16,171.64,5.06,10510.0,17.27,39639.0,9.94
10,POWERGRID.NS,2021-08-15,184.75,8.60,10216.0,22.93,40398.0,8.06
...,...,...,...,...,...,...,...,...
7,ASIANPAINT.NS,2020-11-15,2184.60,8.66,5350.0,23.73,18329.0,92.06
8,ASIANPAINT.NS,2021-02-15,2486.10,12.91,6788.0,28.67,19697.0,86.71
9,ASIANPAINT.NS,2021-05-16,2774.50,8.88,6651.0,32.73,21712.0,84.77
10,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15


In [4]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

<ipython-input-4-e22d75f1c612>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
<ipython-input-4-e22d75f1c612>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
<ipython-input-4-e22d75f1c612>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
6,POWERGRID.NS,2020-08-15,131.55,2.94,9457.0,15.21,38020.0,8.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,POWERGRID.NS,2020-11-15,141.00,4.44,9530.0,15.96,38499.0,8.83,2.94,15.21,9457.0,8.65,131.55,4.93,2.18,7.11,0.77,51.02,7.11
8,POWERGRID.NS,2021-02-15,159.49,4.83,10142.0,16.96,39277.0,9.40,4.44,15.96,9530.0,8.83,141.00,6.27,6.86,13.13,6.42,8.78,13.13
9,POWERGRID.NS,2021-05-16,171.64,5.06,10510.0,17.27,39639.0,9.94,4.83,16.96,10142.0,9.40,159.49,1.83,5.85,7.68,3.63,4.76,7.68
10,POWERGRID.NS,2021-08-15,184.75,8.60,10216.0,22.93,40398.0,8.06,5.06,17.27,10510.0,9.94,171.64,32.77,-25.12,7.65,-2.80,69.96,7.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,ASIANPAINT.NS,2020-11-15,2184.60,8.66,5350.0,23.73,18329.0,92.06,2.28,23.65,2923.0,76.22,1802.55,0.34,20.85,21.19,83.03,279.82,21.19
8,ASIANPAINT.NS,2021-02-15,2486.10,12.91,6788.0,28.67,19697.0,86.71,8.66,23.73,5350.0,92.06,2184.60,20.82,-7.02,13.80,26.88,49.08,13.80
9,ASIANPAINT.NS,2021-05-16,2774.50,8.88,6651.0,32.73,21712.0,84.77,12.91,28.67,6788.0,86.71,2486.10,14.16,-2.55,11.61,-2.02,-31.22,11.61
10,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15,8.88,32.73,6651.0,84.77,2774.50,11.15,-3.44,7.71,-16.03,-33.22,7.71


In [5]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == '2021-11-15'].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
11,TITAN.NS,2021-11-15,115.75,3030.43,38.17
11,SBILIFE.NS,2021-11-15,60.18,10.76,3.19
11,HEROMOTOCO.NS,2021-11-15,55.17,191.11,-3.22
11,HDFCLIFE.NS,2021-11-15,38.97,2.26,7.47
11,BAJAJFINSV.NS,2021-11-15,29.10,34.74,27.64
11,ASIANPAINT.NS,2021-11-15,27.05,4.72,4.50
11,UPL.NS,2021-11-15,24.10,-6.32,-1.21
11,RELIANCE.NS,2021-11-15,19.77,11.47,20.84
11,BAJAJ-AUTO.NS,2021-11-15,18.63,74.31,-4.99
11,LT.NS,2021-11-15,18.54,54.90,17.48


In [6]:
df_analysis_final.to_csv('data/Nifty_FvS_QoQ_Analysis.csv', index=False)

In [7]:
df_growth = pd.read_csv('data/Nifty_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,POWERGRID.NS,2020-11-15,141.00,4.44,9530.0,15.96,38499.0,8.83,2.94,15.21,9457.0,8.65,131.55,4.93,2.18,7.11,0.77,51.02,7.11
2,POWERGRID.NS,2021-02-15,159.49,4.83,10142.0,16.96,39277.0,9.40,4.44,15.96,9530.0,8.83,141.00,6.27,6.86,13.13,6.42,8.78,13.13
3,POWERGRID.NS,2021-05-16,171.64,5.06,10510.0,17.27,39639.0,9.94,4.83,16.96,10142.0,9.40,159.49,1.83,5.85,7.68,3.63,4.76,7.68
4,POWERGRID.NS,2021-08-15,184.75,8.60,10216.0,22.93,40398.0,8.06,5.06,17.27,10510.0,9.94,171.64,32.77,-25.12,7.65,-2.80,69.96,7.65
5,POWERGRID.NS,2021-11-15,182.00,4.84,10267.0,23.33,41135.0,7.80,8.60,22.93,10216.0,8.06,184.75,1.74,-3.28,-1.54,0.50,-43.72,-1.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,ASIANPAINT.NS,2020-11-15,2184.60,8.66,5350.0,23.73,18329.0,92.06,2.28,23.65,2923.0,76.22,1802.55,0.34,20.85,21.19,83.03,279.82,21.19
296,ASIANPAINT.NS,2021-02-15,2486.10,12.91,6788.0,28.67,19697.0,86.71,8.66,23.73,5350.0,92.06,2184.60,20.82,-7.02,13.80,26.88,49.08,13.80
297,ASIANPAINT.NS,2021-05-16,2774.50,8.88,6651.0,32.73,21712.0,84.77,12.91,28.67,6788.0,86.71,2486.10,14.16,-2.55,11.61,-2.02,-31.22,11.61
298,ASIANPAINT.NS,2021-08-15,2988.70,5.93,5585.0,36.38,24374.0,82.15,8.88,32.73,6651.0,84.77,2774.50,11.15,-3.44,7.71,-16.03,-33.22,7.71


In [8]:
df_growth[(df_growth['Sales%'] > 5) & (df_growth['EPS%'] > 5)]['Stock'].value_counts()[0:20]

JSWSTEEL.NS      4
TATASTEEL.NS     4
LT.NS            3
NTPC.NS          3
ULTRACEMCO.NS    3
BAJAJ-AUTO.NS    3
ONGC.NS          3
BPCL.NS          3
GRASIM.NS        3
NESTLEIND.NS     3
SHREECEM.NS      3
ITC.NS           2
ADANIPORTS.NS    2
CIPLA.NS         2
TITAN.NS         2
COALINDIA.NS     2
HEROMOTOCO.NS    2
DIVISLAB.NS      2
RELIANCE.NS      2
ASIANPAINT.NS    2
Name: Stock, dtype: int64

In [9]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [10]:
df_tmp_final[(df_tmp_final['Sales%'] > 5) & (df_tmp_final['EPS%'] > 5)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
11,JSWSTEEL.NS,2021-11-15,12.46,21.46,-10.47,0,0,0.0,1.0,1.0
71,HINDALCO.NS,2021-11-15,15.25,22.66,6.04,0,0,0.0,1.0,1.0
47,TATASTEEL.NS,2021-11-15,12.95,33.73,-11.99,0,0,0.0,1.0,1.0
287,INFY.NS,2021-11-15,6.12,5.66,3.97,0,0,0.0,3.0,3.0
155,BAJFINANCE.NS,2021-11-15,6.39,47.74,23.11,0,0,4.0,4.0,8.0
119,RELIANCE.NS,2021-11-15,19.77,11.47,20.84,0,0,4.0,5.0,9.0
179,DRREDDY.NS,2021-11-15,17.03,161.65,4.10,0,0,3.0,6.0,9.0
23,NTPC.NS,2021-11-15,8.42,5.40,15.43,0,0,6.0,4.0,10.0
89,LT.NS,2021-11-15,18.54,54.90,17.48,0,0,4.0,6.0,10.0
161,BPCL.NS,2021-11-15,7.74,11.48,-5.93,0,0,4.0,6.0,10.0


In [11]:
df_growth[df_growth['Stock'] == 'HDFCBANK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
229,HDFCBANK.NS,2020-11-15,-3.68,10.94,32.40
230,HDFCBANK.NS,2021-02-15,-3.39,13.71,15.48
231,HDFCBANK.NS,2021-05-16,-3.03,-3.89,-12.32
232,HDFCBANK.NS,2021-08-15,0.96,-6.34,10.04
233,HDFCBANK.NS,2021-11-15,1.39,14.65,1.77


In [12]:
df_growth[df_growth['Stock'] == 'ICICIBANK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
223,ICICIBANK.NS,2020-11-15,-3.54,47.19,34.75
224,ICICIBANK.NS,2021-02-15,-4.87,12.43,33.00
225,ICICIBANK.NS,2021-05-16,-4.39,-11.18,-7.74
226,ICICIBANK.NS,2021-08-15,1.14,-2.69,17.98
227,ICICIBANK.NS,2021-11-15,0.25,27.62,10.17


In [13]:
df_growth[df_growth['Stock'] == 'AXISBANK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
277,AXISBANK.NS,2020-11-15,-8.44,53.85,40.69
278,AXISBANK.NS,2021-02-15,-6.02,-28.33,22.38
279,AXISBANK.NS,2021-05-16,-2.12,123.26,-8.72
280,AXISBANK.NS,2021-08-15,3.59,-19.90,10.93
281,AXISBANK.NS,2021-11-15,2.76,43.69,-3.13


In [14]:
df_growth[df_growth['Stock'] == 'INDUSINDBK.NS'][cols]

,Stock,Quarter,Sales%,EPS%,Return%
289,INDUSINDBK.NS,2020-11-15,1.19,19.16,51.91
290,INDUSINDBK.NS,2021-02-15,-1.64,25.09,33.34
291,INDUSINDBK.NS,2021-05-16,1.30,9.21,-13.26
292,INDUSINDBK.NS,2021-08-15,3.24,9.60,14.96
293,INDUSINDBK.NS,2021-11-15,-0.47,12.80,1.25


In [15]:
df_tmp_final.sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
11,JSWSTEEL.NS,2021-11-15,12.46,21.46,-10.47,0,0,0.0,1.0,1.0
71,HINDALCO.NS,2021-11-15,15.25,22.66,6.04,0,0,0.0,1.0,1.0
47,TATASTEEL.NS,2021-11-15,12.95,33.73,-11.99,0,0,0.0,1.0,1.0
287,INFY.NS,2021-11-15,6.12,5.66,3.97,0,0,0.0,3.0,3.0
167,HCLTECH.NS,2021-11-15,2.93,1.69,4.74,0,0,0.0,3.0,3.0
185,TCS.NS,2021-11-15,3.21,6.86,1.82,0,0,0.0,4.0,4.0
41,DIVISLAB.NS,2021-11-15,1.38,8.81,-2.12,0,0,2.0,2.0,4.0
257,WIPRO.NS,2021-11-15,6.51,-9.63,7.27,0,1,0.0,5.0,5.0
293,INDUSINDBK.NS,2021-11-15,-0.47,12.80,1.25,1,0,7.0,0.0,7.0
155,BAJFINANCE.NS,2021-11-15,6.39,47.74,23.11,0,0,4.0,4.0,8.0
